In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [7]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.8, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD-2l_momentum.txt", "w")

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.7946
Validation accuracy: 0.7852, validation loss: 1.2079
Epoch 2
Training loss: 0.8806
Validation accuracy: 0.8393, validation loss: 0.6935
Epoch 3
Training loss: 0.5906
Validation accuracy: 0.8636, validation loss: 0.5442
Epoch 4
Training loss: 0.4846
Validation accuracy: 0.8747, validation loss: 0.4753
Epoch 5
Training loss: 0.4301
Validation accuracy: 0.8822, validation loss: 0.4360
Epoch 6
Training loss: 0.3964
Validation accuracy: 0.8867, validation loss: 0.4102
Epoch 7
Training loss: 0.3734
Validation accuracy: 0.8906, validation loss: 0.3916
Epoch 8
Training loss: 0.3561
Validation accuracy: 0.8926, validation loss: 0.3777
Epoch 9
Training loss: 0.3425
Validation accuracy: 0.8943, validation loss: 0.3655
Epoch 10
Training loss: 0.3312
Validation accuracy: 0.8978, validation loss: 0.3561
Epoch 11
Training loss: 0.3216
Validation accuracy: 0.9002, validation loss: 0.3474
Epoch 12
Training loss: 0.3131
Validation accuracy: 0.9012, validation loss: 0.3397
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9374, device='cuda:0'), tensor(0.2233, device='cuda:0'))
Valid: (tensor(0.9292, device='cuda:0'), tensor(0.2554, device='cuda:0'))


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9418, device='cuda:0'), tensor(0.2076, device='cuda:0'))
Valid: (tensor(0.9331, device='cuda:0'), tensor(0.2396, device='cuda:0'))


In [14]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [15]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [16]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9418, device='cuda:0'), tensor(0.2076, device='cuda:0'))
Valid: (tensor(0.9331, device='cuda:0'), tensor(0.2396, device='cuda:0'))


## Epoch average

In [17]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [18]:
log_file = open("SGD-2l_momentum-avg.txt", "w")
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.8, weight_decay=1e-5, mode="epoch_avg")

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.7747
Validation accuracy: 0.7963, validation loss: 1.1880
Epoch 2
Training loss: 0.8673
Validation accuracy: 0.8440, validation loss: 0.6869
Epoch 3
Training loss: 0.5856
Validation accuracy: 0.8628, validation loss: 0.5413
Epoch 4
Training loss: 0.4831
Validation accuracy: 0.8740, validation loss: 0.4755
Epoch 5
Training loss: 0.4299
Validation accuracy: 0.8801, validation loss: 0.4368
Epoch 6
Training loss: 0.3971
Validation accuracy: 0.8837, validation loss: 0.4121
Epoch 7
Training loss: 0.3743
Validation accuracy: 0.8880, validation loss: 0.3942
Epoch 8
Training loss: 0.3573
Validation accuracy: 0.8916, validation loss: 0.3787
Epoch 9
Training loss: 0.3439
Validation accuracy: 0.8936, validation loss: 0.3673
Epoch 10
Training loss: 0.3326
Validation accuracy: 0.8966, validation loss: 0.3574
Epoch 11
Training loss: 0.3229
Validation accuracy: 0.8992, validation loss: 0.3497
Epoch 12
Training loss: 0.3147
Validation accuracy: 0.9009, validation loss: 0.3414
E

In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9367, device='cuda:0'), tensor(0.2258, device='cuda:0'))
Valid: (tensor(0.9286, device='cuda:0'), tensor(0.2586, device='cuda:0'))


In [21]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [22]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [23]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9321, device='cuda:0'), tensor(0.2463, device='cuda:0'))
Valid: (tensor(0.9228, device='cuda:0'), tensor(0.2774, device='cuda:0'))


In [24]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [25]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
  (4): LogSoftmax(dim=-1)
)

In [26]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9321, device='cuda:0'), tensor(0.2463, device='cuda:0'))
Valid: (tensor(0.9228, device='cuda:0'), tensor(0.2774, device='cuda:0'))
